In [1]:
# 3D world coordinate system:
#     - face x
#     - left y
#     - up z


#                     z                      
#                     |                    
#                     |                   
#                     |                  
#                     |     x            
#                     |    /              
#                     |   /                 
#                     |  /                   
#                     | /                    
#    y________________|/                      
#                       o          

# 2D map coordinate system
#   agent pose dafiinition: (x, y, theta:[0, 2pi])
#
#         x1
#    0________________x                      
#    |    |               
#    |    |              
# y1 |----a --------             
#    |      -           
#    |        -     theta     
#    |          -          
#    |            >         
#    |                      
#    y
#

# Simple map - world coordinate system conversion
#
# void Costmap2D::mapToWorld(unsigned int mx, unsigned int my, double & wx, double & wy) const
# {
#   wx = origin_x_ + (mx + 0.5) * resolution_;
#   wy = origin_y_ + (my + 0.5) * resolution_;
# }

# bool Costmap2D::worldToMap(double wx, double wy, unsigned int & mx, unsigned int & my) const
# {
#   mx = static_cast<unsigned int>((wx - origin_x_) / resolution_);
#   my = static_cast<unsigned int>((wy - origin_y_) / resolution_);
# }


# octomap
#          z            
#         /              
#        /                 
#       /                   
#      /                    
#     /________________ x                     
#    | o          
#    |                      
#    |                    
#    |                   
#    y                  


# GridMap in SLAM

#         map_x
#       0________________z                      
#       |    |               
#       |    |              
#map_y  |----a --------             
#       |      -           
#       |        -     theta     
#       |          -          
#       |            >         
#       |                      
#       x
#

# example

In [2]:
from HomeAssis.mapping.object_map_aux.cg_agent_aux import matrix_to_quaternion, build_rotation
import math
import torch
import numpy as np

In [3]:
# matrix to rotate around z-axis with x degree
import math
def get_matrix(x):
    return [
            [math.cos(x), -math.sin(x), 0], 
            [math.sin(x), math.cos(x), 0], 
            [0, 0, 1]
            ]

matrix = get_matrix(math.radians(190))
matrix

[[-0.984807753012208, 0.17364817766693047, 0],
 [-0.17364817766693047, -0.984807753012208, 0],
 [0, 0, 1]]

In [4]:
a = matrix_to_quaternion(torch.tensor(np.array(matrix)))
a

tensor([-0.0872,  0.0000,  0.0000,  0.9962], dtype=torch.float64)

In [5]:
build_rotation(a[None, :])

tensor([[[-0.9848,  0.1736,  0.0000],
         [-0.1736, -0.9848,  0.0000],
         [ 0.0000,  0.0000,  1.0000]]], device='cuda:0')

In [6]:
def get_yaw(q):
    qw, qx, qy, qz = q[0], q[1], q[2], q[3]
    sqw = qw * qw
    sqx = qx * qx
    sqy = qy * qy
    sqz = qz * qz

    sarg = -2 * (qx*qz - qw*qy) / (sqx + sqy + sqz + sqw)
    if sarg <= -0.99999:
        yaw = -2 * math.atan2(qy, qx)
    elif sarg >= 0.99999:
        yaw = 2 * math.atan2(qy, qx)
    else:
        yaw = math.atan2(2 * (qx*qy + qw*qz), sqw + sqx - sqy - sqz)
    
    return yaw

math.degrees(get_yaw(a))

-170.0

In [12]:
def yaw_to_quat(yaw):
  halfYaw = yaw * 0.5
  return (math.cos(halfYaw), 0, 0, math.sin(halfYaw))


In [13]:
a, get_yaw(a), math.degrees(get_yaw(a))

(tensor([-0.0872,  0.0000,  0.0000,  0.9962], dtype=torch.float64),
 -2.9670597283903604,
 -170.0)

In [14]:
yaw_to_quat(get_yaw(a))

(0.08715574274765814, 0, 0, -0.9961946980917455)

In [16]:
tmp = get_yaw(yaw_to_quat(get_yaw(a)))
np.degrees(tmp), tmp

(-170.0, -2.9670597283903604)